In [1]:
import geopandas as gpd
import pandas as pd
from osgeo import gdal
import numpy as np
from osgeo import ogr, osr

# Rwanda

In [18]:
rwa_bridges = gpd.read_file('data/rwa-feb032023.geojson')
rwa_bridges = rwa_bridges[rwa_bridges.geometry.notnull()]
rwa_bridges = rwa_bridges.set_geometry('geometry')
rwa_bridges = rwa_bridges.to_crs(epsg=4326)

# only including bridges that are complete or under construction
rwa_bridges = rwa_bridges[rwa_bridges['Stage'].isin(['Complete', 'Under Construction'])]

rwa_bridges

,Opportunity Name,Level 2 Government,Bridge Name,Project Code,Stage,Sub-Stage,CaseSafeID,B2P Fiscal Year,Individuals Directly Served,Population Estimate 2000m,geometry
2,Rwanda - Nyaruhombo - 1009280,Huye,Nyaruhombo,1009280,Complete,In Service,006f100000aWV5fAAG,2023,2373.0,5115.0,POINT (29.67439 -2.41201)
3,Rwanda - Birambo - 1037670,Gicumbi,Birambo,1037670,Under Construction,,0063Z00000pof1dQAA,2023,500.0,NaN,POINT (30.13857 -1.58050)
4,Rwanda - Ruharazi - 1009311,Nyaruguru,Ruharazi,1009311,Complete,In Service,006f100000asEzNAAU,2019,NaN,7393.0,POINT (29.62627 -2.77676)
11,Rwanda - Kucyaruseke - RW127,Nyaruguru,Kucyaruseke,RW127,Complete,In Service,006f100000Zs7KqAAJ,2018,3000.0,4923.0,POINT (29.55727 -2.71176)
24,Rwanda - Mujerenge - 1007463,Nyamagabe,Mujerenge,1007463,Complete,In Service,006f100000a86F1AAI,2022,995.0,4330.0,POINT (29.42767 -2.45196)
...,...,...,...,...,...,...,...,...,...,...,...
1494,Rwanda - Gahira - 1039372,Muhanga,Gahira,1039372,Complete,,0063Z00000sfo3WQAQ,2023,1200.0,NaN,POINT (29.79893 -1.83072)
1495,Rwanda - Kabutimbo - 1015693,Gakenke,Kabutimbo,1015693,Complete,In Service,006f100000fHqCoAAK,2021,1280.0,NaN,POINT (29.66057 -1.62119)
1500,Rwanda - Rufuha - 1017673,Muhanga,Rufuha,1017673,Complete,In Service,0063Z00000gLr35QAC,2020,1000.0,NaN,POINT (29.76389 -2.01217)
1501,Rwanda - Rubete - 1037177,Nyarugenge,Rubete,1037177,Complete,In Service,0063Z00000plEZkQAM,2022,3937.0,NaN,POINT (30.04886 -2.00275)


in the EPSG 4326 coordinate system, 1 degree = 69 miles. We want to calculate a 2 mile buffer around each point. 2 miles / 69 miles per degree ≈ 0.0289855072463768 degrees

In [ ]:
rwa_bridges['geometry'] = rwa_bridges['geometry'].buffer(0.029)

Population data was procured from World Pop 2020 data. This was converted from raster to vector points using QGIS raster pixels to points. 

In [11]:
# read in rwa_pop_2020.geojson
rwa_pop = gpd.read_file('unsynced-data/rwa-pop-2020.geojson')
# rename pop- population
rwa_pop = rwa_pop.rename(columns={'pop-': 'population'})

In [ ]:
# calculate the sum of rwa_pop points within each of the rwa_bridges polygons and add this to a new column in rwa_bridges
rwa_bridges['pop_sum'] = rwa_bridges['geometry'].apply(lambda x: rwa_pop['population'].loc[rwa_pop.within(x)].sum())
rwa_bridges['pop_sum'] = rwa_bridges['pop_sum'].astype(int)

In [22]:
rwa_bridges

,Opportunity Name,Level 2 Government,Bridge Name,Project Code,Stage,Sub-Stage,CaseSafeID,B2P Fiscal Year,Individuals Directly Served,Population Estimate 2000m,geometry,pop_sum
2,Rwanda - Nyaruhombo - 1009280,Huye,Nyaruhombo,1009280,Complete,In Service,006f100000aWV5fAAG,2023,2373.0,5115.0,"POLYGON ((29.70339 -2.41201, 29.70325 -2.41485...",17644
3,Rwanda - Birambo - 1037670,Gicumbi,Birambo,1037670,Under Construction,,0063Z00000pof1dQAA,2023,500.0,NaN,"POLYGON ((30.16757 -1.58050, 30.16743 -1.58335...",8871
4,Rwanda - Ruharazi - 1009311,Nyaruguru,Ruharazi,1009311,Complete,In Service,006f100000asEzNAAU,2019,NaN,7393.0,"POLYGON ((29.65527 -2.77676, 29.65513 -2.77960...",17113
11,Rwanda - Kucyaruseke - RW127,Nyaruguru,Kucyaruseke,RW127,Complete,In Service,006f100000Zs7KqAAJ,2018,3000.0,4923.0,"POLYGON ((29.58627 -2.71176, 29.58613 -2.71461...",11883
24,Rwanda - Mujerenge - 1007463,Nyamagabe,Mujerenge,1007463,Complete,In Service,006f100000a86F1AAI,2022,995.0,4330.0,"POLYGON ((29.45667 -2.45196, 29.45653 -2.45480...",15898
...,...,...,...,...,...,...,...,...,...,...,...,...
1494,Rwanda - Gahira - 1039372,Muhanga,Gahira,1039372,Complete,,0063Z00000sfo3WQAQ,2023,1200.0,NaN,"POLYGON ((29.82793 -1.83072, 29.82779 -1.83357...",12735
1495,Rwanda - Kabutimbo - 1015693,Gakenke,Kabutimbo,1015693,Complete,In Service,006f100000fHqCoAAK,2021,1280.0,NaN,"POLYGON ((29.68957 -1.62119, 29.68943 -1.62403...",19364
1500,Rwanda - Rufuha - 1017673,Muhanga,Rufuha,1017673,Complete,In Service,0063Z00000gLr35QAC,2020,1000.0,NaN,"POLYGON ((29.79289 -2.01217, 29.79275 -2.01501...",13749
1501,Rwanda - Rubete - 1037177,Nyarugenge,Rubete,1037177,Complete,In Service,0063Z00000plEZkQAM,2022,3937.0,NaN,"POLYGON ((30.07786 -2.00275, 30.07772 -2.00559...",124039


In [23]:
rwa_bridges.to_file('data/rwa-population-2mi.geojson', driver='GeoJSON')
rwa_bridges.to_csv('data/rwa-population-2mi.csv')

# Uganda

In [34]:
uga_bridges = gpd.read_file('data/uga-may242023.geojson')
# uga_bridges = uga_bridges[rwa_bridges.geometry.notnull()]
uga_bridges = uga_bridges.set_geometry('geometry')
uga_bridges = uga_bridges.to_crs(epsg=4326)

# only including bridges that are complete or under construction
uga_bridges = uga_bridges[uga_bridges['Stage'].isin(['Complete', 'Under Construction'])]

uga_bridges

,Opportunity Name,Level 2 Government,Bridge Name,Project Code,Stage,Sub-Stage,B2P Fiscal Year,Individuals Directly Served,Population Estimate 2000m,geometry
7,Uganda - Masaka-munsiso - 1022724,Kabarole,Masaka-munsiso,1022724,Complete,In Service,2022,11100.0,NaN,POINT (30.48413 0.86235)
18,Uganda - Kishanda-kisegere - 1039081,Kanungu,Kishanda-kisegere,1039081,Complete,In Service,2023,2090.0,NaN,POINT (29.65713 -0.93691)
38,Uganda - Kyabayenze - 1015937,Kasese,Kyabayenze,1015937,Complete,In Service,2021,23416.0,NaN,POINT (29.74135 0.08837)
50,Uganda - Kasumbelya - 1013817,Bundibugyo,Kasumbelya,1013817,Complete,NaN,2022,8218.0,NaN,POINT (30.00045 0.64200)
52,Uganda - Tuyobei - 1011084,Bukwa,Tuyobei,1011084,Complete,In Service,2020,1709.0,1066.0,POINT (34.72611 1.43919)
72,Uganda - Mihani-Karichadi - 1014455,Kasese,Mihani-Karichadi,1014455,Complete,In Service,2022,5195.0,NaN,POINT (30.13907 0.46404)
82,Uganda - Kikuyu - 1011474,Bulambuli,Kikuyu,1011474,Complete,In Service,2020,2521.0,5742.0,POINT (34.31263 1.27324)
90,Uganda - Sanzara - 1010460,Kapchorwa,Sanzara,1010460,Complete,In Service,2020,1730.0,1880.0,POINT (34.33432 1.37366)
106,Uganda - Kawuswo - 1011085,Bukwa,Kawuswo,1011085,Complete,In Service,2020,1374.0,27.0,POINT (34.72236 1.41914)
158,Uganda - Kama - UG101,Mbale,Kama,UG101,Complete,In Service,2018,5280.0,16243.0,POINT (34.25246 1.11633)


In the EPSG 4326 coordinate system, 1 degree = 69 miles. We want to calculate a 2 mile buffer around each point. 2 miles / 69 miles per degree ≈ 0.0289855072463768 degrees

In [35]:
uga_bridges['geometry'] = uga_bridges['geometry'].buffer(0.029)

/var/folders/3v/8xxt4zw96bn2l6qwb8w7_5r40000gn/T/ipykernel_44667/58340433.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  uga_bridges['geometry'] = uga_bridges['geometry'].buffer(0.029)


Population data was procured from World Pop 2020 data. This was converted from raster to vector points using QGIS raster pixels to points. 

In [ ]:
# read in rwa_pop_2020.geojson
uga_pop = gpd.read_file('unsynced-data/rwa-pop-2020.geojson')

In [ ]:
# calculate the sum of rwa_pop points within each of the rwa_bridges polygons and add this to a new column in rwa_bridges
uga_bridges['pop_sum'] = uga_bridges['geometry'].apply(lambda x: rwa_pop['population'].loc[uga_pop.within(x)].sum())
uga_bridges['pop_sum'] = uga_bridges['pop_sum'].astype(int)

In [ ]:
# Previewing data
uga_bridges

In [ ]:
uga_bridges.to_file('data/rwa-population-2mi.geojson', driver='GeoJSON')
uga_bridges.to_csv('data/rwa-population-2mi.csv')